In [1]:
#Ingest data
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import os
from six.moves import xrange
import tensorflow as tf
import re
import sys
import tarfile
from six.moves import urllib


In [22]:
IMAGE_SIZE = 24
NUM_CLASSES = 10
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 10000

FLAGS = tf.app.flags.FLAGS
#Basic model params
tf.app.flags.DEFINE_integer('batch_size', 128, """Number of images to process per batch""")
tf.app.flags.DEFINE_string('data_dir', '/opt/cifar10_data', """Path to the CIFAR data directory""")
tf.app.flags.DEFINE_boolean('use_fp16', False, """Train the model using fp16""")
tf.app.flags.DEFINE_string('train_dir', '/opt/cifar10_train',
                           """Directory where to write event logs """
                           """and checkpoint.""")
tf.app.flags.DEFINE_integer('max_steps', 1000000,
                            """Number of batches to run.""")
tf.app.flags.DEFINE_boolean('log_device_placement', False,
                            """Whether to log device placement.""")


ArgumentError: argument --batch_size: conflicting option string(s): --batch_size

In [3]:
def read_cifar10(filename_queue):
    class CIFAR10Record(object):
        pass
    result = CIFAR10Record()
    
    label_bytes = 1
    result.height = 32
    result.width = 32
    result.depth = 3
    image_bytes = result.height * result.width * result.depth
    record_bytes = label_bytes + image_bytes
    reader = tf.FixedLengthRecordReader(record_bytes = record_bytes)
    result.key, value = reader.read(filename_queue)
    record_bytes = tf.decode_raw(value, tf.uint8)
    result.label = tf.cast(tf.strided_slice(record_bytes, [0], [label_bytes]), tf.int32)
    depth_major = tf.reshape(tf.strided_slice(record_bytes, [label_bytes], [label_bytes + image_bytes]),
                            [result.width, result.height, result.depth])
    result.unit8image = tf.transpose(depth_major, [1,2,0])
    return result

In [4]:
def _generate_image_and_label_batch(image, label, min_queue_examples, 
                                   batch_size, shuffle):
    num_preprocess_threads = 16
    if shuffle:
        images, label_batch = tf.train.shuffle_batch([image, label],
                                                    batch_size = batch_size,
                                                    num_threads = num_preprocess_threads,
                                                    capacity = min_queue_examples * 3 * batch_size)
    else:
        images, label_batch = tf.train.batch([image, label],
                                            batch_size = batch_size,
                                            num_threads = num_preprocess_threads,
                                            capacity = min_queue_examples * 3 * batch_size)
    tf.image_summary('images', images)
    return images, tf.reshape(label_batch, [batch_size])

In [5]:
def distorted_inputs(data_dir, batch_size):
    filenames = [os.path.join(data_dir, 'data_batch_%d.bin' % i) for i in xrange(1,6)]
    for f in filenames:
        if not tf.gfile.Exists(f):
            raise ValueError("Failed to find file: " + f)
    filename_queue = tf.train.string_input_producer(filenames)
    read_input = read_cifar10(filename_queue)
    reshaped_image = tf.cast(read_input.unit8image, tf.float32)
    height = IMAGE_SIZE
    width = IMAGE_SIZE
    distorted_image = tf.random_crop(reshaped_image, [height, width, 3])
    distorted_image = tf.image.random_flip_left_right(distorted_image)
    distorted_image = tf.image.random_brightness(distorted_image, max_delta = 63)
    distorted_image = tf.image.random_contrast(distorted_image, lower = 0.2, upper = 1.8)
    float_image = tf.image.per_image_standardization(distorted_image)
    float_image.set_shape([height, width, 3])
    read_input.label.set_shape([1])
    min_fraction_of_examples = 0.4
    min_queue_examples = int(NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN * min_fraction_of_examples)
    print ('Filling queue with %d CIFAR Images before beginning to train', 'This will take a few mins'
          % min_queue_examples)
    return _generate_image_and_label_batch(float_image, read_input.label, min_queue_examples,
                                          batch_size, shuffle = True)

In [6]:
def inputs(eval_data, data_dir, batch_size):
    if not eval_data:
        filenames = [os.path.join(data_dir, 'data_batch_%d,bin' % i) for i in xrange(1, 6)]
        num_examples_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN
    else:
        filenames = [os.path.join(data_dir, 'test_batch_%d,bin' % i) for i in xrange(1, 6)]
        num_examples_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_EVAL
    for f in filenames:
        if not tf.gfile.Exists(f):
            raise ValueError('Failed to find file:' + f)
    filename_queue = tf.train.string_input_producer(filenames)
    read_input = read_cifar10(filename_queue)
    reshaped_image = tf.cast(read_input.unit8image, tf.float32)
    height = IMAGE_SIZE
    width = IMAGE_SIZE
    resized_image = tf.image.resize_image_with_crop_or_pad(reshaped_image, width, height)
    float_image = tf.image_per_standardization(resized_image)
    float_image.set_shape([height, width, 3])
    read_input.label.set_shape([1])
    min_fraction_of_examples_in_queue = 0.4
    min_queue_examples = int(num_examples_per_epoch * min_fraction_of_examples_in_queue)
    return _generate_image_and_label_batch(float_image, read_input.label, 
                                           min_queue_examples, batch_size, shuffle = False)
        

In [7]:
#Constants for the training process
MOVING_AVERAGE_DECAY = 0.9999
NUM_EPOCHS_PER_DECAY = 350.0
LEARNING_RATE_DECAY_FACTOR = 0.1
INITIAL_LEARNING_RATE = 0.1
TOWER_NAME = 'tower'
DATA_URL = 'http://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz'

In [8]:
def _activation_summary(x):
    tensor_name = re.sub('%s_[0-9]*/' % TOWER_NAME, '', x.op.name)
    tf.histogram_summary(tensor_name + '/activations', x)
    tf.scalar_summary(tensor_name + '/sparsity', 
                     tf.nn.zero_fraction(x))
    

In [9]:
def _variable_on_cpu(name, shape, initializer):
    with tf.device('/cpu:0'):
        dtype = tf.float16 if FLAGS.use_fp16 else tf.float32
        var = tf.get_variable(name, shape, initializer = initializer, dtype = dtype)
    return var

In [10]:
def _variable_with_weight_decay(name, shape, stddev, wd):
    dtype = tf.float16 if FLAGS.use_fp16 else tf.float32
    var = _variable_on_cpu(name, shape, tf.truncated_normal_initializer(stddev = stddev, dtype = dtype))
    if wd is not None:
        weight_decay = tf.mul(tf.nn.l2loss(var), wd, name = 'weight_loss')
        tf.add_to_collection('losses', weight_decay)
    return var

In [11]:
def distorted_inputs_main():
    if not FLAGS.data_dir:
        raise ValueError("Please supply a data dir")
    data_dir = os.path.join(FLAGS.data_dir, 'cifar-10-batches-bin')
    images, labels = distorted_inputs(data_dir = data_dir, 
                                     batch_size = FLAGS.batch_size)
    if FLAGS.use_fp16:
        images = tf.cast(tf.float16)
        labels = tf.cast(tf.float16)
    return images, labels     

In [13]:
def inputs_main(eval_data):
    if not FLAGS.data_dir:
        raise ValueError("Please supply a data dir")
    data_dir = os.path.join(FLAGS.data_dir, 'cifar-10-batches-bin')
    images, labels = inputs(eval_data = eval_data,
                            data_dir = data_dir, 
                            batch_size = FLAGS.batch_size)
    if FLAGS.use_fp16:
        images = tf.cast(tf.float16)
        labels = tf.cast(tf.float16)
    return images, labels
    

In [14]:
#Build the CIFAR10 model
def inference(images):
    #Conv1
    with tf.variable_scope('conv1') as scope:
        kernel = _variable_with_weight_decay('weights',
                                            shape = [5, 3, 3, 64], 
                                            stddev = 5e-2,
                                            wd = 0.0)
        conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding = 'SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.0))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv1)
    
    #pool1
    pool1 = tf.nn.max_pool(conv1, ksize = [1, 3, 3, 1], strides = [1, 2, 2, 1],
                              padding = 'SAME', name = 'pool1')
    norm1 = tf.nn.lrn(pool1, 4, bias = 1.0, alpha = 0.001 / 9.0, beta = 0.75, name = 'norm1')
    
    #conv2
    with tf.variable_scope('conv2') as scope:
        kernel = _variable_with_weight_decay('weights',
                                            shape = [5, 5, 64, 64], 
                                            stddev = 5e-2,
                                            wd = 0.0)
        conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding = 'SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.1))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv2)
    
    norm2 = tf.nn.lrn(conv2, 4, bias = 1.0, alpha=0.001 / 9.0, beta=0.75, name = 'norm2')
    pool2 = tf.nn.max_pool(norm2, ksize = [1, 3, 3, 1], strides = [1, 2, 2, 1], padding = 'SAME'
                            , name = 'pool2')
    
    #local3
    with tf.variable_scope('local3') as scope:
        reshape = tf.reshape(pool2, [FLAGS.batch_size, -1])
        dim = reshape.get_shape()[1].value
        weights = _variable_with_weight_decay('weights', shape=[dim, 384],
                                          stddev=0.04, wd=0.004)
        biases = _variable_on_cpu('biases', [384], tf.constant_initializer(0.1))
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name = scope.name)
        _activation_summary(local3)
    
    #local4
    with tf.variable_scope('local4') as scope:
        weights = _variable_with_weight_decay('weights', shape=[384, 192],
                                          stddev=0.04, wd=0.004)
        biases = _variable_on_cpu('biases', [192], tf.constant_initializer(0.1))
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name = scope.name)
        _activation_summary(local4)
        
    #linear
    with tf.variable_scope('softmax_linear') as scope:
        weights = _variable_with_weight_decay('weights', [192, NUM_CLASSES],
                                          stddev=1/192.0, wd=0.0)
        biases = _variable_on_cpu('biases', [NUM_CLASSES], tf.constant_initializer(0.0))
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name = scope.name)
        _activation_summary(softmax_linear)
    return softmax_linear
        
    
        
        

In [15]:
def loss(logits, labels):
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,
                                                                  logits = logits,
                                                                  name = "cross_entropy_per_example")
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name = 'cross_entropy')
    tf.add_to_collection(losses, cross_entropy_mean)
    return tf.add_n(tf.get_collection('losses'), name = 'total_loss')


In [16]:
def _add_loss_summaries(total_loss):
    loss_averages = tf.train.ExponentialMovingAverage(0.9, name = 'avg')
    losses = tf.get_collection('losses')
    loss_averages_op = loss_averages.apply(losses + [total_loss])
    for l in losses + [total_loss]:
        tf.scalar_summary(l.op.name + ' (raw)', l)
        tf.scalar_summary(l.op.name, loss_averages.average(l))
    return loss_averages_op
    

In [17]:
def train(total_loss, global_step):
    num_batches_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN / FLAGS.batch_size
    decay_steps = int(num_batches_per_epoch * NUM_EPOCHS_PER_DECAY)
    lr = tf.train.exponential_decay(INITIAL_LEARNING_RATE,
                                   global_step,
                                   decay_steps,
                                   LEARNING_RATE_DECAY_FACTOR,
                                   staircase = True)
    tf.scalar_summary('learning_rate', lr)
    loss_averages_op = _add_loss_summaries(total_loss)
    
    #compute gradients
    with tf.control_dependencies([loss_averages_op]):
        opt = tf.train.GradientDescentOptimizer(lr)
        grads = opt.compute_gradients(total_loss)
    
    apply_gradient_op = opt.apply_gradients(grads, global_step = global_step)
    
    #Add histograms for trainable variables
    for grad, var in grads:
        if grad is not None:
            tf.histogram_summary(var.op.name + '/gradients', grad)
    
    #Track the moving averages of all trainable variables
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,
                                                         global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    
    with tf.control_dependencies([apply_gradient_op, variables_averages_op]):
        train_op = tf.no_op(name='train')
    
    return train_op
    

In [18]:
def maybe_download_and_extract():
    dest_directory = FLAGS.data_dir
    if not os.path.exists(dest_directory):
        os.makedirs(dest_directory)
    filename = DATA_URL.split('/')[-1]
    filepath = os.path.join(dest_directory, filename)
    if not os.path.exists(file_path):
        def _progress(count, block_size, total_size):
            sys.stdout.write('\r>> Downloading %s %.1f%%' % (filename,
          float(count * block_size) / float(total_size) * 100.0))
        sys.stdout.flush()
        filepath, _ = urllib.request.urlretrieve(DATA_URL, filepath, _progress)
        print()
        statinfo = os.stat(filepath)
        print('Successfully downloaded', filename, statinfo.st_size, 'bytes.')

    tarfile.open(filepath, 'r:gz').extractall(dest_directory)
    

In [19]:
#Train the model
#FLAGS = tf.app.flags.FLAGS


In [20]:
def train_main():
    with tf.Graph().as_default():
        global_step = tf.contrib.framework.get_or_create_global_step()
        images, labels = distroted_inputs_main()
        logits = inference(images)
        loss = loss(logits, labels)
        train_op = train_main(loss, global_step)
        
        class _LoggerHook(tf.train.SessionRunHook):
            def begin(self):
                self._step = -1
            
            def before_run(self, run_context):
                self._step += 1
                self.start_time = time.time()
                return tf.train.SessionRunArgs(loss)
            
            def after_run(self, run_context, run_values):
                duration = time.time() - self._start_time
                loss_value = run_values.results
                if self._step % 10 == 0:
                    num_examples_per_step = FLAGS.batch_size
                    examples_per_sec = num_examples_per_step /duration
                    sec_per_batch = float(duration)

                    format_str = ('%s: step %d, loss = %.2f (%.1f examples/sec; %.3f '
                        'sec/batch)')
                    print (format_str % (datetime.now(), self._step, loss_value,
                               examples_per_sec, sec_per_batch))
       
    with tf.train.MonitoredTrainingSession(
        checkpoint_dir = FLAGS.train_dir,
        hooks=[tf.train.StopAtStepHook(last_step=FLAGS.max_steps),
               tf.train.NanTensorHook(loss),
               _LoggerHook()],
        config=tf.ConfigProto(
            log_device_placement=FLAGS.log_device_placement)) as mon_sess:
        while not mon_sess.should_stop():
            mon_sess.run(train_op)
            
                
        

In [21]:
#Run the model
maybe_download_and_extract()

NameError: global name 'FLAGS' is not defined